In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import StandardScaler
from bed_reader import open_bed, sample_file

In [11]:
def simulator(X, sigma):
    '''
    Simulate phenotype y from X and sigmas
    '''
    N = X.shape[0]
    M = X.shape[1]
    y = np.zeros((N,1))
    sigma_epsilon=1 - sigma # environmental effect sizes
    betas = np.random.randn(M,1)*np.sqrt(sigma) # additive SNP effect sizes
    y += X@betas/np.sqrt(M)
    #print(f'sigma_epislon={sigma_epsilon}')
    y += np.random.randn(N,1)*np.sqrt(sigma_epsilon) # add the effect sizes
    return y, betas

def solve_linear_equation(X, y):
    '''
    Solve least square
    '''
    sigma = np.linalg.lstsq(X, y, rcond=None)[0]
    return sigma


def solve_linear_qr(X, y):
    '''
    Solve least square using QR decomposition
    '''
    Q, R = scipy.linalg.qr(X)
    sigma = scipy.linalg.solve_triangular(R, np.dot(Q.T, y))
    return sigma

def RHE(X,y,num_random_vect=10,seed=1,verbose=False):
    '''
    RHE estimation
    '''
    np.random.seed(seed)
    N = X.shape[0]
    M = X.shape[1]
    T = np.zeros((2,2))
    q = np.zeros((2,1))

    Xi = X.copy()/np.sqrt(M)
    for _ in range(num_random_vect):
        # Generate random vector to estimate trace
        rand_vector = np.random.randn(N,1)
        T[0,0] += rand_vector.T@Xi@Xi.T@Xi@Xi.T@rand_vector/num_random_vect

    T[1,0] = np.trace(Xi@Xi.T)
    T[0,1] = T[1,0]
    T[1,1] = N

    q[0] = y.T@Xi@Xi.T@y
    q[1] = y.T@y

    if verbose:
        print(T)
    sigma_est = solve_linear_equation(T,q)
    return sigma_est


def HE (X, y, verbose = False):
    '''
    HE estimation (without using random vectors)
    '''
    N = X.shape[0]
    M = X.shape[1]
    T = np.zeros((2,2))
    q = np.zeros((2,1))

    Xi = X.copy()/np.sqrt(M)
    T[0,0] = np.trace(Xi@Xi.T@Xi@Xi.T)
    T[1,0] = np.trace(Xi@Xi.T)
    T[0,1] = T[1,0]
    T[1,1] = N

    q[0] = y.T@Xi@Xi.T@y
    q[1] = y.T@y

    if verbose:
        print(T)

    sigma_est = solve_linear_equation(T,q)
    return sigma_est

## In the cell below, we simulate the genotype matrix by taking random samples of the MAF for each SNP from a uniform distribution. Then for each individual (based on the MAF of that SNP), run a Bernoulli trial to get the genotype value at that location (i.e., whether the individual has the SNP or not; either 0,1 or 2)
I'll also share the genotype matrices for you to benchmark RHE and the python version (here), but this a very simple simulation. For instance, here we don't consider the LD structure here at all.

In [12]:
M=100 # M = 500000
N=10000  # N=300000
target_index=1
sigmas_list=[0.3]
# t0 = time()
for i in range(1):
    np.random.seed(i)
    sigmas=sigmas_list[i]

    MAFs = np.random.uniform(0.01,0.5,M)

    P_matrix = np.tile(MAFs, (N, 1))

    X = np.random.binomial(2, P_matrix) # X here is the un-standardized, raw genotype matrix (entries are 0 or 1)

    X = StandardScaler().fit_transform(X) # standardize X

    y, beta_list = simulator(X, sigmas) # generate phenotype with some beta values

    print(f'Actual sigmas are: {sigmas}')
    sigma_est=RHE(X,y,num_random_vect=10,seed=42) # run py-RHE
    sigma_est2=HE(X, y)

    print ('RHE estimated sigmas are:',*sigma_est, '\nHE estimated sigmas are:', *sigma_est2)

Actual sigmas are: 0.3
RHE estimated sigmas are: [0.27646048] [0.71014217] 
HE estimated sigmas are: [0.28039846] [0.7062042]


## And here's an (almost) identical code but instead of simulating genotype, it reads in the genotype file (.bed) using the bed-reader. One major difference is, in real genotype, there is often missingness, which needs to be imputed.
### Here's one way of imputing: simply replace the missing values by the average of non-missing entries for that SNP. This is different than what's done in the RHE code.


In [13]:
'''
impute the genotype matrix for any missingness by using the average of other samples
this function also standardizes it
'''
def impute_geno(X):
    N = X.shape[0]
    M = X.shape[1]
    X_imp = X.copy()
    for m in range(M):
        cnt = 0
        csum = 0
        for n in range(N):
            if not np.isnan(X[n, m]):
                cnt += 1
                csum += X[n,m]
        csum /= cnt
        X_imp[:,m] = np.nan_to_num(X_imp[:,m], nan=csum*0.5)
    X_imp = (X_imp-np.mean(X_imp, axis=0))/np.std(X_imp, axis=0)
    return X_imp

In [15]:
geno_path="/content/drive/MyDrive/Colab Notebooks/RHE_project/geno/1kindv_1ksnps.bed"
bed = open_bed(geno_path)
X = bed.read()
print(X.shape)

FileNotFoundError: ignored

In [ ]:
X_imp = impute_geno(X)

In [ ]:
sigma = 0.2
y, beta_list = simulator(X_imp, sigma) # generate phenotype with some beta values

print(f'Actual sigmas are: {sigma}')
sigma_est=RHE(X_imp,y,num_random_vect=10,seed=42) # run py-RHE
sigma_est2=HE(X_imp, y) # run exact-HE

print ('RHE estimated sigmas are:',*sigma_est, '\nHE estimated sigmas are:', *sigma_est2)

Actual sigmas are: 0.2
RHE estimated sigmas are: [0.16427573] [0.831989] 
HE estimated sigmas are: [0.16982968] [0.82643505]


## TODO: In the "geno" directory, you will find several genotype (.bed) files with multiple combinations of N (samples) and M (snps). As discussed in our meeting:

## 1. Benchmark the python version of RHE & HE and the original, C++ version on those genotypes files and plot the run time (y-axis) against NxM (x-axis), and interpolate to large samples (UKBB) of 300k individuals x 454k SNPs.

## 2. Repeat the same, but using the simulated genotypes
For simulated phenotypes, to run it on the C++ RHE, you'd have to port the matrix into PLINK bed. Read https://zzz.bwh.harvard.edu/plink/binary.shtml for how the encoding works. You might also want to take a look at this documentation for packages that does it: https://rdrr.io/cran/genio/man/write_bed.html

## 3*.  If you are able to do these and get some sensible results, one extra task is to take a look at how RHE imputes & standardizes the genotype matrix, and see if you can implement it in python